# Inspecting the empirical datasets (1/n): Fitting -- the Bacteria dataset

1. Prefilter: Drop genes present in <=5% species)
2. EM run x 10: lmax=2, ncat=3; default CLI (same procedure as [a cross validation test](../fit-to-real-data/240701-1-fitting-main-bacteria.ipynb))
3. Find the best run

In [1]:
import gzip
import json
import shutil
from multiprocessing import Pool
from operator import attrgetter, itemgetter, methodcaller
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from colaml import *
from colaml.misc import parsimony
from colaml.__main__ import phytbl_from_json, model_from_json
from myconfig import DATASET_DIR, DATA_DIR, ROOT_DIR
from mydata import filter_table

In [2]:
from threadpoolctl import threadpool_limits
threadpool_limits(1, user_api='blas')

In [3]:
lmax, ncat = 2, 3
WORK_DIR = DATA_DIR/'inspect-real-data'/'bacteria'

## Data preproc

In [4]:
dataset_path = DATASET_DIR/'02-bacteria'/'bacteria-main-v0.json.gz'
phytbl, columns = filter_table(
    *phytbl_from_json(dataset_path, lmax), max_missing=0.05, adjust_branch=True
)

In [5]:
with gzip.open(WORK_DIR/f'bacteria-lmax{lmax}-filt05.json.gz', 'wt') as file:
    json.dump({
        'tree': phytbl.tree.to_ete3().write(format=3, format_root_node=True), 
        'OGs' : pd.DataFrame.from_dict(
            phytbl.to_dict(), orient='index', columns=columns
        ).to_dict(orient='split')
    }, file, indent=2)

## Run EM 10 times

In [6]:
root_seed = 42
jobs = pd.DataFrame([
    (i, WORK_DIR/'bacteria-lmax2-filt05.json.gz', WORK_DIR/f'bacteria-lmax2-filt05.fit{i:02d}.json.gz')
    for i in range(1, 11)
], columns=['repid', 'input_file', 'output_file'])

def colaml_ezbatch(job):
    seed = np.random.default_rng([root_seed, job.repid]).integers(2**16)
    !set -x && colaml fit mmm -i {job.input_file} -o {job.output_file} \
                              --lmax {lmax} --ncat {ncat} --seed {seed} -q
    
with Pool(10) as pool:
    for _ in tqdm(pool.imap(colaml_ezbatch, map(itemgetter(1), jobs.iterrows()))):
        pass

0it [00:00, ?it/s]

+ colaml fit mmm -i /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.json.gz -o /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.fit02.json.gz --lmax 2 --ncat 3 --seed 14556 -q
+ colaml fit mmm -i /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.json.gz -o /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.fit01.json.gz --lmax 2 --ncat 3 --seed 41811 -q
+ colaml fit mmm -i /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.json.gz -o /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.fit07.json.gz --lmax 2 --ncat 3 --seed 31513 -q
+ colaml fit mmm -i /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.json.gz -o /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.fit10.json.gz --lmax 2 --ncat 3 --seed 55403 -q
+ colaml fit mmm -i /home/jovyan/data/inspect-real-data/bacteria/bacteria-lmax2-filt05.json.gz -o /home/jovyan/data/inspect-real-data/bacteria/bacte

## Find best run

In [7]:
result = jobs.assign(
    loglik = lambda df:
        df['output_file']
            .apply(model_from_json)                          # path -> mmm
            .apply(methodcaller('sufficient_stats', phytbl)) # mmm + phytable -> stats
            .apply(attrgetter('col_loglik'))                 # stats.col_loglik
            .apply(sum)
).sort_values(by='loglik', ascending=False)

result

,repid,input_file,output_file,loglik
0,1,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488735
2,3,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488738
5,6,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488741
7,8,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488766
4,5,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488830
9,10,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488893
1,2,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488909
3,4,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488946
6,7,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488957
8,9,/home/jovyan/data/inspect-real-data/bacteria/b...,/home/jovyan/data/inspect-real-data/bacteria/b...,-434621.488962


In [8]:
shutil.copyfile(
    result.iloc[0]['output_file'], 
    ROOT_DIR/'results'/'bacteria-lmax2-filt05.bestfit.json.gz'
)

PosixPath('/home/jovyan/results/bacteria-lmax2-filt05.bestfit.json.gz')